In [ ]:
// Mes imports de librairies externes

USE {
    repositories {
        //google()
        // à exécuter en dehors du proxy sinon KO
        mavenCentral()
        //maven("https://artifactory.mycloud.intrabpce.fr/artifactory/g-android-maven-proxy/")
        //maven("https://artifactory.mycloud.intrabpce.fr/artifactory/maven-third-party-android-libs/")
    }
    dependencies {
        val ktor_version = "2.2.4"
        implementation("io.ktor:ktor-client-apache-jvm:$ktor_version")
        implementation("io.ktor:ktor-serialization-kotlinx-json:$ktor_version")
    }
}

In [ ]:

import io.ktor.client.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import kotlinx.coroutines.runBlocking

val client = HttpClient()
runBlocking {
    client.get("https://google.com").bodyAsText()
 }

In [ ]:
import io.ktor.client.engine.apache.*
import io.ktor.client.*
import io.ktor.client.plugins.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import io.ktor.http.*
import kotlinx.serialization.SerialName
import kotlinx.serialization.Serializable
import kotlinx.serialization.decodeFromString
import kotlinx.serialization.json.Json

@Serializable
data class SearchProductList(
    @SerialName("products") val products: List<ProductDetail>? = null
)

@Serializable
data class Product(
    @SerialName("code") val productId: String? = null,
    @SerialName("product") val productDetail: ProductDetail? = null,
)

@Serializable
data class ProductDetail(
    @SerialName("code") val code: String = "",
    @SerialName("product_name_fr") val label: String = "",
    @SerialName("ingredients_text") val ingredientsText: String = "",
    @SerialName("ingredients_text_fr") val ingredientsTextFr: String = "",
    @SerialName("generic_name_fr") val genericName: String = "",
    @SerialName("image_url") val imageUrl: String = "",
    @SerialName("nutrition_grade_fr") val nutriScore: String = "",
    @SerialName("ecoscore_grade") val ecoScore: String = "",
    @SerialName("ecoscore_score") val ecoScoreValue: String = ""
)

class NutriScoreApp() {
    val jsonFormatter = Json { ignoreUnknownKeys = true }

    suspend fun getProduct(productId: String): Product {
        val client = getHttpClient()
        val getProductUrl = "https://fr.openfoodfacts.org/api/v0/produit/$productId.json"
        val productText = client.get(getProductUrl + FIELDS).bodyAsText()
        val product: Product = jsonFormatter.decodeFromString(productText)
        client.close()
        return product
    }

    suspend fun searchProductsJson(search: String): String {
        val client = getHttpClient()
        val searchProductUrl = "https://ssl-api.openfoodfacts.org/cgi/search.pl?search_simple=1&amp;json=1&amp;action=process&amp;sort_by=unique_scans_n"
        val extraParams = "&amp;search_terms=${search.encodeURLPath()}&amp;page=1"
        val searchText = client.get(searchProductUrl + FIELDS + extraParams).bodyAsText()
        return searchText
    }

    suspend fun searchProducts(search: String): List<ProductDetail> {
        val searchResult: SearchProductList = jsonFormatter.decodeFromString(searchProductsJson(search))
        client.close()
        return searchResult.products.orEmpty()
    }

    private fun getHttpClient() = HttpClient {
            install(HttpTimeout) {
                requestTimeoutMillis = 30000
            }
        }

    companion object {
        const val FIELDS = "&amp;fields=selected_images,image_url,product_name,brands,quantity,code,nutrition_grade_fr,ecoscore_grade,product_name_fr,nova_groups,allergens_tags,additives_tags,ingredients_from_palm_oil_n,carbon-footprint-from-known-ingredients_product,carbon-footprint-from-meat-or-fish_product,nutriments"
    }
}

In [ ]:
import kotlinx.coroutines.runBlocking

// Exemple de produit :
// Gâteaux au chocolat : 3175681134935
runBlocking {
    val nutriScoreApp = NutriScoreApp()
    val product = nutriScoreApp.getProduct("3175681134935")
    HTML(
        "<h2>${product.productDetail?.label}</h2><br>" +
                "<img src='${product.productDetail?.imageUrl}'/>"
    )
}


In [ ]:
import kotlinx.coroutines.runBlocking

// Exemple de recherche :
runBlocking {
    val search = "pate à tartiner noisette"
    val nutriScoreApp = NutriScoreApp()
    val searchResult = nutriScoreApp.searchProducts(search)
    println(searchResult)

    var searchResultToDisplay: String = ""
    searchResult.forEach {
        searchResultToDisplay += "<h2>${it.label}</h2><br>" +
        if (it.imageUrl.isNotEmpty()) "<img src='${it.imageUrl}'/>" else ""
    }
    HTML(searchResultToDisplay)
}

In [48]:
%useLatestDescriptors
%use lets-plot
%use krangl


In [ ]:
import kotlinx.coroutines.runBlocking
import krangl.util.colSelectAsNames

// Exemple de
runBlocking {
    val search = "pate à tartiner noisette"
    val nutriScoreApp = NutriScoreApp()
    val searchResultJson: String = nutriScoreApp.searchProductsJson(search)

    val df = DataFrame.fromJsonString(searchResultJson)
    df.head(3)
    //println(df.cols.get(9))

    val ecoscores = df.cols.get(9).toStrings().sort()//listOf("a", "b", "c", "d", "e") to df.cols.get(9).toStrings().sort()
    val nutriscores = df.cols.get(14).toStrings().sort()//listOf("a", "b", "c", "d", "e") to df.cols.get(14).toStrings().sort()
    val names = df.cols.get(16)

    val plotData = mapOf<String, Any>("x" to ecoscores, "y" to nutriscores)

    val plot = letsPlot(plotData) + geomPoint(
        color = "dark-green",
        size = 4.0
    ) { x = "x"; y = "y" }
    plot.show()

    //val xNutriscore = df

    /*df.letsPlot {

     }*/


    /*val plotResult = letsPlot(searchResult)
    searchResult.plot {

    }*/

}